# Исключения в Python

Основное, что надо знать про исключения - это конструкции языка, предназначенные для работы с ними:

```python
try:
    raise KeyboardInterrupt('Just an msg for arg[0]', 'Just an msg for arg[1]')
except KeyboardInterrupt as kbrd_ex:
    print(kbrd_ex.args[0], kbrd_ex.args[1])
except (ValueError, ZeroDivisionError) as err:
    print(err)
else:
    print('No exception has happened')
finally:
    print('Goodbye people!')
```

Говоря более формально и полно, исключения обрабатываются следующим образом:

1. Входим в блок, подразумевающий возникновение исключений, через конструкцию `try:`  
2. Обрабатываем исключения в блоках `except:`  
3. Исключение вызывается конструкцией `raise`. Важно понимать, что она может вызываться не только в блоке `try`, но и в блоке `except`, причем в блоке `except` мы можем вызывать ее без всяких параметров, пробрасывая выше по стеку исключение, сгенерированное в `try`. Если мы хотим в блоке `except` на основании одного исключения сгенерировать другое, то надо пользоваться конструкцией, аналогичной `raise TypeException('some info') from err`, где `err` - это объект сгенерированного ранее исключения.
3. Если надо как-то отдельно обрабатывать ситуацию, когда исключений не возникло, то используем `else:`  
4. Если надо выполнить одинаковые действия и когда исключение было и когда его не было, используем `finally:`  
5. В обработчике можно получить доступ к объекту исключения через конструкцию, аналогичную `except KeyboardInterrupt as kbrd_ex:`  
6. В объекты исключений можно добавлять свои параметры. Читать их надо через атрибут `args`.  
7. В одном обработчике можно обрабатывать исключения разных типов через конструкцию, аналогичную `except (ValueError, ZeroDivisionError) as err:`.
8. Если мы хотим проверять выполнение каких-то условий, то пригодится конструкция, аналогичная `assert var >= 0, 'Значение должно быть неотрицательным!'`. При невыполнении заданного условия будет сгенерировано исключение `AssertionError`. Можно избежать генерирования исключений `AssertionError`, запустив `python.exe` с параметром `-O`. 

Иногда нам может потребоваться посмотреть стек вызовов в момент исключения. Сделать это можно так:

In [49]:
import traceback

try:
    1 / 0
except ZeroDivisionError:
    print(traceback.print_exc())

None


Traceback (most recent call last):
  File "C:\Users\kb255048\AppData\Local\Temp/ipykernel_23476/921786945.py", line 4, in <module>
    1 / 0
ZeroDivisionError: division by zero


Теперь посмотрим на иерархию классов исключений стандартной библиотеки Python:

In [42]:
def get_class_hierarchy(base_cls: type, intendation: int = 0, clss = []) -> str:
    if len(clss) > 0 and base_cls.__name__ not in clss:
        return ''
    result = '\n' * (intendation > 0) + '-' * intendation + base_cls.__name__
    if len(base_cls.__subclasses__()) != 0:
        for subclass in base_cls.__subclasses__():
            result += get_class_hierarchy(subclass, intendation + 3, clss)
    return result

print(get_class_hierarchy(base_cls=BaseException))

BaseException
---Exception
------TypeError
---------MultipartConversionError
---------FloatOperation
------StopAsyncIteration
------StopIteration
------ImportError
---------ModuleNotFoundError
---------ZipImportError
------OSError
---------ConnectionError
------------BrokenPipeError
------------ConnectionAbortedError
------------ConnectionRefusedError
------------ConnectionResetError
---------------RemoteDisconnected
---------BlockingIOError
---------ChildProcessError
---------FileExistsError
---------FileNotFoundError
---------IsADirectoryError
---------NotADirectoryError
---------InterruptedError
------------InterruptedSystemCall
---------PermissionError
---------ProcessLookupError
---------TimeoutError
---------UnsupportedOperation
---------Error
------------SameFileError
---------SpecialFileError
---------ExecError
---------ReadError
---------herror
---------gaierror
---------timeout
---------SSLError
------------SSLCertVerificationError
------------SSLZeroReturnError
------------S

Здесь надо отметить, что все классы являются потомками класса `BaseException`. При этом его непосредственными потомками является довольно малое число классов, в то время, как основное число классов-исключений являются потомками класса `Exception`. От него же надо наследовать и пользовательские классы исключений.

In [48]:
try:
    a = 1
    b = 'str'
    c = a/b
except (ZeroDivisionError, TypeError) as err:
    print(err)

unsupported operand type(s) for /: 'int' and 'str'


Важно понимать, что исключения сильно замедляют производительность. Рассмотрим пример.

In [50]:
%%timeit

my_dict = {'foo': 1}
for _ in range(1000):
    try:
        my_dict['bar']
    except KeyError:
        #Писать pass в except - это очень плохая практика, но здесь не практика =)
        pass

142 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [51]:
%%timeit

my_dict = {'foo': 1}
for _ in range(1000):
    if 'bar' in my_dict:
        _ = my_dict['bar']

20.3 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


То есть, код без исключений отрабатывает в 7 раз быстрее - 20 микросекунд на итерацию против 140.

Напишем простую программу, которая считает число строк в текстовом файле

In [8]:
%%writefile wc.py

import sys

def count_wc(filename):
    result = 0
    with open(filename) as f:
        for _ in f:
            result += 1
    return result

def process_file(filename):
    wc = count_wc(filename)
    print(f'File {filename} contains {wc} line(s)')

def main():
    process_file(sys.argv[1])

if __name__ == '__main__':
    main()

Overwriting wc.py


Убедимся в работоспособности нашей программы:

In [9]:
%%bash

python3 wc.py 'wc.py'

File wc.py contains 19 line(s)


А вызов кода ниже приведет к исключению:

```bash
%%bash

python3 wc.py 'no_such_file.py'
```

Исключение будет выглядить примерно так:

```bash
-bash: line 1: fg: no job control
Traceback (most recent call last):
  File "wc.py", line 19, in <module>
    main()
  File "wc.py", line 16, in main
    process_file(sys.argv[1])
  File "wc.py", line 12, in process_file
    wc = count_wc(filename)
  File "wc.py", line 6, in count_wc
    with open(filename) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'no_such_file.py'
```

# Задания

## Задание 1

Первое задание на этой неделе — не сложное, для разогрева. Ваша задача: написать python-модуль **solution.py**, внутрь которого необходимо поместить код класса **FileReader**. Конструктор этого класса принимает один параметр: путь до файла на диске. В классе **FileReader** должен быть реализован метод **read**, возвращающий строку - содержимое файла, путь к которому был указан при создании экземпляра класса. Python модуль должен быть написан таким образом, чтобы импорт класса **FileReader** из него не вызвал ошибок.

При написании реализации метода **read**, вам нужно учитывать случай, когда при инициализации был передан путь к несуществующему файлу. Требуется обработать возникающее при этом исключение **FileNotFoundError** и вернуть из метода read пустую строку.

In [54]:
%%writefile solution.py

class FileReader:
    def __init__(self, file_name: str):
        self.file_name = file_name
        
    def read(self):
        result = None
        try:
            f = open(self.file_name)
            result = f.read()
        except FileNotFoundError:
            result = ''
        return result

Overwriting solution.py


In [58]:
from solution import FileReader
reader = FileReader('not_exist_file.txt')
text = reader.read()
print(text)
with open('some_file.txt', 'w') as f:
    f.write('some text...')
reader = FileReader('some_file.txt')
text = reader.read()
print(text)
print(type(reader))


some text...
<class 'solution.FileReader'>


## Задание 2

Как правило задачи про классы не носят вычислительный характер. Обычно нужно написать классы, которые отвечают определенным интерфейсам. Насколько удобны эти интерфейсы и как сильно связаны классы между собой, определит легкость их использования в будущих программах.

Предположим есть данные о разных автомобилях и спецтехнике. Данные представлены в виде таблицы с характеристиками. Вся техника разделена на три вида: спецтехника, легковые и грузовые автомобили. Обратите внимание на то, что некоторые характеристики присущи только определенному виду техники. Например, у легковых автомобилей есть характеристика «кол-во пассажирских мест», а у грузовых автомобилей — габариты кузова: «длина», «ширина» и «высота».

<table>
    <tr>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">тип (car_type)</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">марка (brand)</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">кол-во пассажирских мест (passenger_seats_count)</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">фото (photo_file_name)</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">кузов дxшxв, м (body_whl)</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">грузоподъемность, тонн (carrying)</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">дополнительно (extra)</span></td>
    </tr>
    <tr>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">car</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">nissan xttrail</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">4</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">f1.jpeg</span></td>
        <td></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">2.5</span></td>
        <td></td>
    </tr>
    <tr>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">truck</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">man</span></td>
        <td></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">f2.jpeg</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">8x3x2.5</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">20</span></td>
        <td></td>
    </tr>
    <tr>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">car</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">mazda 6</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">4</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">f3.jpeg</span></td>
        <td></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">2.5</span></td>
        <td></td>
    </tr>
    <tr>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">spec_machine</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">hitachi</span></td>
        <td></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">f4.jpeg</span></td>
        <td></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">1.2</span></td>
        <td><span style="color: rgb(31, 31, 31); font-family: opensans, arial, sans-serif; font-size: 12.6px; white-space: pre-wrap;">легкая техника для уборки снега</span></td>
    </tr>
</table>

Вам необходимо создать свою иерархию классов для данных, которые описаны в таблице. Классы должны называться **CarBase** (базовый класс для всех типов машин), **Car** (легковые автомобили), **Truck** (грузовые автомобили) и **SpecMachine** (спецтехника). Все объекты имеют обязательные атрибуты:

- **car_type**, значение типа объекта и может принимать одно из значений: «car», «truck», «spec_machine».

- **photo_file_name**, имя файла с изображением машины, допустимы названия файлов изображений с расширением из списка: «.jpg», «.jpeg», «.png», «.gif»

- **brand**, марка производителя машины

- carrying, грузоподъемность

В базовом классе **CarBase** нужно реализовать метод **get_photo_file_ext** для получения расширения файла изображения. Расширение файла можно получить при помощи os.path.splitext.

Для грузового автомобиля необходимо в конструкторе класса определить атрибуты: **body_length**, **body_width**, **body_height**, отвечающие соответственно за габариты кузова — длину, ширину и высоту. Габариты передаются в параметре **body_whl** (строка, в которой размеры разделены латинской буквой **«x»**). Обратите внимание на то, что характеристики кузова должны быть вещественными числами и характеристики кузова могут быть не валидными (например, пустая строка). В таком случае всем атрибутам, отвечающим за габариты кузова, присваивается значение равное нулю.

Также для класса грузового автомобиля необходимо реализовать метод **get_body_volume**, возвращающий объем кузова.

В классе Car должен быть определен атрибут passenger_seats_count (количество пассажирских мест), а в классе SpecMachine — extra (дополнительное описание машины).

Полная информация о атрибутах классов приведена в таблице ниже, где **1** - означает, что атрибут обязателен для объекта, **0** - атрибут должен отсутствовать.

<table><tbody><tr><td></td><td>Car</td><td>Truck</td><td>SpecMachine</td></tr><tr><td>car_type</td><td>1</td><td>1</td><td>1</td></tr><tr><td>photo_file_name</td><td>1</td><td>1</td><td>1</td></tr><tr><td>brand</td><td>1</td><td>1</td><td>1</td></tr><tr><td>carrying</td><td>1</td><td>1</td><td>1</td></tr><tr><td>passenger_seats_count</td><td>1</td><td>0</td><td>0</td></tr><tr><td>body_width</td><td>0</td><td>1</td><td>0</td></tr><tr><td>body_height</td><td>0</td><td>1</td><td>0</td></tr><tr><td>body_length</td><td>0</td><td>1</td><td>0</td></tr><tr><td>extra</td><td>0</td><td>0</td><td>1</td></tr></tbody></table>

Обратите внимание, что у каждого объекта из иерархии должен быть свой набор атрибутов и методов. Например, у класса легковой автомобиль не должно быть метода get_body_volume в отличие от класса грузового автомобиля. Имена атрибутов и методов должны совпадать с теми, что описаны выше.

Далее вам необходимо реализовать функцию **get_car_list**, на вход которой подается имя файла в формате **csv**. Файл содержит данные, аналогичные строкам из таблицы. Вам необходимо прочитать этот файл построчно при помощи модуля стандартной библиотеки **csv**. Затем проанализировать строки на валидность и создать список объектов с автомобилями и специальной техникой. Функция должна возвращать список объектов.

Первая строка в исходном файле — это заголовок **csv**, который содержит имена колонок. Нужно пропустить первую строку из исходного файла. Обратите внимание на то, что в некоторых строках исходного файла , данные могут быть заполнены некорректно, например, отсутствовать обязательные поля или иметь не валидное значение. В таком случае нужно проигнорировать подобные строки и не создавать объекты. Строки с пустым или не валидным значением для **body_whl** игнорироваться не должны.  Вы можете использовать стандартный механизм обработки исключений в процессе чтения, валидации и создания объектов из строк csv-файла. Проверьте работу вашего кода с входным файлом, прежде чем загружать задание для оценки.

Пример кода, демонстрирующего чтение csv файла:

```python
import csv

with open(csv_filename) as csv_fd:
    reader = csv.reader(csv_fd, delimiter=';')
    next(reader)  # пропускаем заголовок
    for row in reader:
        print(row)
```

Решение задания:

In [118]:
%%writefile solution.py

import re
import csv
import os

class CarBase:
    
    car_type, brand, photo_file_name, carrying = [property()] * 4
    
    def __init__(self, car_type, brand, photo_file_name, carrying):
        self.car_type = car_type
        self.brand = brand
        self.photo_file_name = photo_file_name
        self.carrying = carrying
    
    @car_type.getter
    def car_type(self):
        return self._car_type
    
    @car_type.setter
    def car_type(self, value):
        self._car_type = value if value.lower() in ['car', 'truck', 'spec_machine'] else ''
    
    @brand.getter
    def brand(self):
        return self._brand
    
    @brand.setter
    def brand(self, value):
        self._brand = value
        
    @photo_file_name.getter
    def photo_file_name(self):
        return self._photo_file_name
    
    @photo_file_name.setter
    def photo_file_name(self, value):
        self._photo_file_name = value
        
    @carrying.getter
    def carrying(self):
        return self._carrying
    
    @carrying.setter
    def carrying(self, value):
        self._carrying = value if value > 0 else 0.0
        
    def get_photo_file_ext(self):
        return re.search(r'\.jpe?g$|\.png$|\.gif', value).group()

class Car(CarBase):
    
    passenger_seats_count = property()
    
    def __init__(self, brand, photo_file_name, carrying, passenger_seats_count):
        super().__init__('car', brand, photo_file_name, carrying)
        self.passenger_seats_count = passenger_seats_count
    
    @passenger_seats_count.getter
    def passenger_seats_count(self):
        return self._passenger_seats_count
    
    @passenger_seats_count.setter
    def passenger_seats_count(self, value):
        self._passenger_seats_count = value if value >= 0 else 0

class Truck(CarBase):
    body_width, body_height, body_length = [property()] * 3
    
    def __init__(self, brand, photo_file_name, carrying, body_whl):
        super().__init__('truck', brand, photo_file_name, carrying)
        if not re.match('/\d*\.\d+x\d\.\dx\d\.\d/gm', body_whl):
            self.body_width, self.body_height, self.body_length = [0.0] * 3
        else:
            self.body_width, self.body_height, self.body_length = [float(alnum) for alnum in body_whl.lower().split('x')]
            
    @body_width.getter
    def body_width(self):
        return self._body_width
    
    @body_width.setter
    def body_width(self, value):
        self._body_width = value if value >= 0 else 0.0
        
    @body_height.getter
    def body_height(self):
        return self._body_height
    
    @body_height.setter
    def body_height(self, value):
        self._body_height = value if value >= 0 else 0.0
        
    @body_length.getter
    def body_length(self):
        return self._body_length
    
    @body_length.setter
    def body_length(self, value):
        self._body_length = value if value >= 0 else 0.0
        
    def get_body_volume(self):
        return self.body_width * self.body_height * self.body_length

class SpecMachine(CarBase):
    def __init__(self, brand, photo_file_name, carrying, extra):
        super().__init__('spec_machine', brand, photo_file_name, carrying)
        self.extra = extra

def get_car_list(csv_filename):
    car_list = []  
    if not os.path.exists(csv_filename):
        return car_list
    car_type, brand, passenger_seats_count, photo_file_name, body_whl, carrying, extra = list(range(7))
    constructors = {'car': (lambda row: Car(row[brand], row[photo_file_name], float(row[carrying]), int(row[passenger_seats_count]))),\
                    'truck': (lambda row: Truck(row[brand], row[photo_file_name], float(row[carrying]), row[body_whl])),\
                    'spec_machine': (lambda row: Truck(row[brand], row[photo_file_name], float(row[carrying]), row[extra]))}
    with open(csv_filename) as csv_fd:
        reader = csv.reader(csv_fd, delimiter=';')
        for row in reader:
            if row[car_type] not in constructors:
                continue
            print(row)
            car_list.append(constructors[row[car_type]](row))
    return car_list

get_car_list('coursera_week3_cars.csv')

Overwriting solution.py
